# Introduction to the Integrated Deepfake Detection System

## Overview

Welcome to the Integrated Deepfake Detection System! This project is a comprehensive effort to tackle the growing problem of deepfake videos using advanced machine learning techniques. As deepfake technology continues to evolve, it poses significant challenges to privacy, security, and authenticity in media. Our system is designed to detect deepfakes by analyzing multiple aspects of video content, specifically focusing on spatial, temporal, and micro-expression features. By integrating these different feature types, we aim to create a robust and reliable detection mechanism capable of identifying manipulated video content.

## Objectives

The primary objective of this project is to develop a deepfake detection system that can accurately distinguish between genuine and manipulated videos. This involves:

1. **Spatial Feature Extraction**: Analyzing individual frames of a video to capture static facial features. This is accomplished using pre-trained Convolutional Neural Networks (CNNs) such as ResNet50 and VGG16, which are fine-tuned for this task.
   
2. **Temporal Feature Extraction**: Understanding how facial features change over time by analyzing sequences of frames. Bidirectional Long Short-Term Memory (BiLSTM) networks are employed to capture temporal dependencies and detect inconsistencies in the flow of facial expressions.

3. **Micro-Expression Analysis**: Focusing on subtle facial movements that are difficult to replicate in deepfake videos. This module uses specialized CNN architectures to extract and analyze micro-expressions, providing an additional layer of detection.

4. **Feature Fusion**: Combining spatial, temporal, and micro-expression features using attention mechanisms to form a comprehensive feature set that enhances detection accuracy. This fusion approach leverages the strengths of each feature type to make a final decision about the authenticity of the video.

## Why This Approach?

Deepfake detection is a challenging task due to the sophistication of the algorithms used to create these fakes. Traditional detection methods that rely on a single type of feature often fail to capture the complexity of manipulations. By integrating spatial, temporal, and micro-expression features, our approach provides a multi-dimensional analysis that significantly improves the likelihood of detecting deepfakes. This holistic strategy addresses the limitations of existing methods and provides a more reliable solution.

### Key Challenges Addressed:

- **Variability in Deepfake Techniques**: Different deepfake algorithms have varying strengths and weaknesses. By analyzing multiple aspects of the video, our system can detect a wide range of manipulations.
- **Subtle Manipulations**: Some deepfakes are so well-crafted that the manipulations are not immediately noticeable to the human eye. Micro-expression analysis helps in detecting these subtle manipulations.
- **Generalization**: Ensuring that the model is not overfitting to specific datasets or types of deepfakes. The fusion of different feature types helps the model generalize better across diverse datasets.

## Dataset

For this project, we use the **FaceForensics++** dataset, which is a benchmark dataset commonly used in deepfake detection research. It contains a collection of both original and manipulated video sequences, providing a diverse set of examples for training and testing the system. The dataset is divided into two main categories:

- **Original Sequences**: Videos that have not been altered, serving as ground truth for authenticity.
- **Manipulated Sequences**: Videos that have been altered using various deepfake techniques, providing examples of fake content.

## Notebook Structure

This Jupyter Notebook is structured to guide you through the different stages of the deepfake detection process:

1. **Data Preprocessing**: Preparing the video frames for feature extraction, including face detection, alignment, and normalization.
   
2. **Model Architecture**: Detailed implementation of the spatial, temporal, and micro-expression feature extraction models. This section includes building and training the CNN (ResNet50, VGG16), BiLSTM, and micro-expression analysis models.

3. **Feature Fusion and Classification**: Combining the extracted features and using attention mechanisms to improve the detection accuracy. The final output layer provides the classification result, indicating whether the video is genuine or a deepfake.

4. **Evaluation and Results**: Testing the trained model on a set of test videos and evaluating its performance using metrics such as accuracy, precision, recall, and F1-score.

5. **Conclusion and Future Work**: Summarizing the findings and discussing potential improvements and future directions for research in deepfake detection.

## Getting Started

To run this notebook, ensure that you have all the necessary dependencies installed. The required libraries are listed in the `requirements.txt` file. Use the following command to install them:

```bash
pip install -r requirements.txt
```

Additionally, make sure you have the FaceForensics++ dataset downloaded and placed in the appropriate directory as specified in the notebook.

## Conclusion

The Integrated Deepfake Detection System represents a significant step towards addressing the challenges posed by deepfake technology. By leveraging multiple feature extraction methods and incorporating attention mechanisms, this system aims to provide a robust solution for identifying manipulated video content. We hope that this project will contribute to the ongoing efforts to maintain the integrity and authenticity of digital media.

Let's get started and dive into the world of deepfake detection!

## **Data Loading**

In [1]:
import pandas as pd
import numpy as np
import os

## **Preprocessing Phase**

## **Model Building**

## 1. **Facial Feature Extractor Module**

### 1. Spatial Feature Extractor

In [2]:
import numpy as np
import cv2
import os
import tensorflow as tf
from keras.api.layers import TimeDistributed,Input, Flatten, GlobalAvgPool1D
from keras.api.applications.resnet50 import  ResNet50, preprocess_input as resnet_preprocess
from keras.api.models import  Model
import matplotlib.pyplot as plt

In [3]:
base_model = ResNet50(include_top=False, weights='imagenet', pooling='avg', input_shape=(224,224,3))

In [4]:
spatial_model =  Model(inputs=base_model.input, outputs=base_model.output)

In [5]:
for layer in spatial_model.layers:
    layer.trainable = False

In [6]:
spatial_model.output.shape

(None, 2048)

### 2. Temporal Feature Extractor

In [7]:
from tensorflow import keras
from keras.layers import Bidirectional,LSTM,Input
from keras.models import Model

In [8]:
def build_temporal_feature_extractor():
    input_seq = Input(shape=(30,2048))

    lstm_1 = LSTM(128, return_sequences=True, dropout=0.2)
    lstm_2 = LSTM(64,return_sequences=True, dropout=0.2)

    lstm_out = Bidirectional(lstm_1)(input_seq)
    lstm_out = Bidirectional(lstm_2)(lstm_out)

    model = Model(inputs=input_seq, outputs=lstm_out)
    return model

In [9]:
temporal_model = build_temporal_feature_extractor()

In [10]:
temporal_model.output

<KerasTensor shape=(None, 30, 128), dtype=float32, sparse=False, name=keras_tensor_178>

## **2. Micro Expression Inconsistency Detection Module**

### **1. Micro Expression Feature Extraction**

In [11]:
from keras.api.layers import Conv2D,BatchNormalization,Activation,MaxPooling2D,Dropout, Dense, Flatten

In [12]:
def build_micro_exp_spatial_feature_extractor():
    spatial_inputs = Input(shape=(64,64,3))

    # Layer 1
    micro_exp_x = Conv2D(32,(3,3),padding='same')(spatial_inputs)
    micro_exp_x = BatchNormalization()(micro_exp_x)
    micro_exp_x = Activation('relu')(micro_exp_x)
    micro_exp_x = MaxPooling2D(pool_size=(2,2))(micro_exp_x)
    
    # Layer 2
    micro_exp_x = Conv2D(64,(3,3),padding='same')(micro_exp_x)
    micro_exp_x = BatchNormalization()(micro_exp_x)
    micro_exp_x = Activation('relu')(micro_exp_x)
    micro_exp_x = MaxPooling2D(pool_size=(2,2))(micro_exp_x)
    
    # Layer 3
    micro_exp_x = Conv2D(128,(3,3),padding='same')(micro_exp_x)
    micro_exp_x = BatchNormalization()(micro_exp_x)
    micro_exp_x = Activation('relu')(micro_exp_x)
    micro_exp_x = MaxPooling2D(pool_size=(2,2))(micro_exp_x)
    
    # Layer 4
    micro_exp_x = Flatten()(micro_exp_x)
    micro_exp_x = Dense(256, activation='relu')(micro_exp_x)
    # Add dropouts in case of overfitting
    
    micro_exp_output = Dense(128, activation='relu')(micro_exp_x)
    
    micro_exp_spatial_feature_extractor = Model(inputs=spatial_inputs, outputs=micro_exp_output)
    
    return micro_exp_spatial_feature_extractor

In [13]:
# input_shape = (64,64,3)
# inputs = Input(input_shape)
# 
# # Layer 1
# x = Conv2D(32,(3,3),padding='same')(inputs)
# x = BatchNormalization()(x)
# x = Activation('relu')(x)
# x = MaxPooling2D(pool_size=(2,2))(x)
# 
# # Layer 2
# x = Conv2D(64,(3,3),padding='same')(x)
# x = BatchNormalization()(x)
# x = Activation('relu')(x)
# x = MaxPooling2D(pool_size=(2,2))(x)
# 
# # Layer 3
# x = Conv2D(128,(3,3),padding='same')(x)
# x = BatchNormalization()(x)
# x = Activation('relu')(x)
# x = MaxPooling2D(pool_size=(2,2))(x)
# 
# # Layer 4
# x = Flatten()(x)
# x = Dense(256, activation='relu')(x)
# # Add dropouts in case of overfitting
# 
# output = Dense(128, activation='relu')(x)
# 
# micro_exp_feature_extractor = Model(inputs=inputs, outputs=output)
# micro_exp_feature_extractor.compile(optimizer='adam', loss='mean_squared_error', metrics=['accuracy'])
# micro_exp_feature_extractor.summary()

In [14]:
micro_exp_spatial = build_micro_exp_spatial_feature_extractor()

In [15]:
micro_exp_spatial.output

<KerasTensor shape=(None, 128), dtype=float32, sparse=False, name=keras_tensor_194>

### **2. Temporal Inconsistency Detection in Micro Expression**

In [16]:
from keras.api.layers import Attention,Concatenate

In [17]:
def build_temporal_inconsistency_attention():
    temp_inputs = Input(shape=(30,128))
    
    x_mic_exp = Bidirectional(LSTM(128, return_sequences=True, dropout=0.2))(temp_inputs)
    
    x_mic_exp = Bidirectional(LSTM(64, return_sequences=True, dropout=0.2))(x_mic_exp)
    
    attention_output = Attention()([x_mic_exp,x_mic_exp])
    
    x_mic_exp = Dense(256, activation='relu')(attention_output)
    # Add dropout layers for overfitting
    x_mic_exp = Dense(128, activation='relu')(x_mic_exp)
    
    mic_exp_temp_model = Model(inputs=temp_inputs, outputs=x_mic_exp)
    
    mic_exp_temp_model.compile(optimizer='adam', loss='mean_squared_error', metrics=['accuracy'])
    
    return mic_exp_temp_model
    

In [18]:
mic_exp_temp = build_temporal_inconsistency_attention()

In [19]:
mic_exp_temp.output.shape

(None, 30, 128)

## **3. Feature Fusion Layer**

### **1. Spatial Attention Mechanism**

In [20]:
from keras.api.layers import Multiply, GlobalAvgPool1D, GlobalAveragePooling2D, Lambda

In [21]:
def build_spatial_attention_mechanism(feature_maps):
    """
    :param feature_maps: 
    :return: weighted feature maps
    """

    expanded_tensor = Lambda(lambda x: tf.expand_dims(tf.expand_dims(x, axis=1),axis=2))(feature_maps)
    
    attention_map = Conv2D(1, kernel_size=(1,1), strides=(1,1), padding='same')(expanded_tensor)
    
    attention_map = Activation('sigmoid')(attention_map) # 'sigmoid' or 'softmax' can be used as an activation function
    
    # Element wise multiplication of feature_maps and attention_map
    weighted_feature_map = Multiply()([feature_maps, attention_map])
    
    # Convert the weighted feature map into a context vector
    spatial_context_vectors = GlobalAveragePooling2D()(expanded_tensor)
    
    return spatial_context_vectors
    

### **2. Temporal Attention Mechanism**

In [22]:
def build_temporal_attention_mechanism(feature_maps):
    """
    :param feature_maps: 
    :return weighted_feature_maps: 
    """
    
    temporal_attention_scores = Dense(1, activation='tanh')(feature_maps)
    
    temporal_attention_weights = Activation('softmax')(temporal_attention_scores)
    
    weighted_temporal_features = Multiply()([feature_maps, temporal_attention_weights])
    
    context_vector = Lambda(lambda x: tf.reduce_sum(x, axis=1))(weighted_temporal_features)
    
    return context_vector

### **3. Micro Expression Attention Mechanism**

#### **1. Spatial Micro Expression Attention Mechanism**

In [23]:
def build_spatial_micro_expression_attention_mechanism(micro_exp_spatial_feature_maps):
    """
    :param micro_exp_spatial_feature_maps: 
    :return weighted micro_exp feature maps : 
    """

    reshaped_map = Lambda(lambda x: tf.expand_dims(tf.expand_dims(x, axis=1),axis=2))(micro_exp_spatial_feature_maps)
    
    attention_map = Conv2D(1,(1,1),strides=(1,1),padding="same")(reshaped_map)
    
    attention_map = Activation('sigmoid')(attention_map)
    
    weighted_micro_exp_feature_map = Multiply()([micro_exp_spatial_feature_maps,attention_map])
    
    micro_exp_spatial_context_vector = GlobalAveragePooling2D()(weighted_micro_exp_feature_map)
    
    return micro_exp_spatial_context_vector

#### **2. Temporal Micro Expression Attention Mechanism**

In [24]:
def build_temporal_micro_expression_attention_mechanism(micro_exp_feature_vectors):
    """
    :param micro_exp_feature_vectors: 
    :return micro_exp_context_vectors: 
    """
    
    attention_scores = Dense(1,activation='tanh')(micro_exp_feature_vectors)
    
    attention_weights = Activation('softmax')(attention_scores)
    
    weighted_micro_exp_temporal_features = Multiply()([attention_weights, micro_exp_feature_vectors])
    
    micro_exp_context_vector = Lambda(lambda x:tf.reduce_sum(x, axis=1))(weighted_micro_exp_temporal_features)
    
    return micro_exp_context_vector

### **4. Concatenation Layer**

In [25]:
from keras.api.layers import Concatenate

In [26]:
def build_feature_fusion_layer(spatial_features, temporal_features, micro_exp_spatial_features, micro_exp_temporal_features):
    
    spatial_context_vectors = build_spatial_attention_mechanism(feature_maps=spatial_features)

    temporal_context_vector = build_temporal_attention_mechanism(feature_maps=temporal_features)
    
    micro_exp_spatial_context_vector = build_spatial_micro_expression_attention_mechanism(micro_exp_spatial_feature_maps=micro_exp_spatial_features)
    
    micro_exp_temporal_context_vector = build_temporal_micro_expression_attention_mechanism(micro_exp_feature_vectors=micro_exp_temporal_features)
    
    concatenated_feature_vector = Concatenate()([
        spatial_context_vectors,
        temporal_context_vector,
        micro_exp_spatial_context_vector,
        micro_exp_temporal_context_vector
    ])
    
    return concatenated_feature_vector

## **4. Face Swap Detection Model**

In [27]:
def build_face_swap_detection_model(concatenated_feature_vector):
    
    dense_units = [256,128,64]
    
    x_face_swap = concatenated_feature_vector
    for unit in dense_units:
        x_face_swap = Dense(unit, activation='relu')(x_face_swap)
        x_face_swap = Dropout(0.5)(x_face_swap)
    
    op_face_swap = Dense(1,activation='sigmoid')(x_face_swap)
    
    face_swap_detector_model = Model(inputs=concatenated_feature_vector, outputs=op_face_swap)
    
    return face_swap_detector_model    

## **Face Swap Detection Pipeline**

In [28]:
temporal_input = Input(shape=(30,2048))
micro_exp_spatial_input = build_micro_exp_spatial_feature_extractor().input
micro_exp_temporal_input = Input(shape=(30,128))


In [29]:
spatial_features = spatial_model.output

expanded_spatial_features = Lambda(lambda x: tf.expand_dims(x, axis=1))(spatial_features)
reshaped_spatial_features = Lambda(lambda x: tf.reshape(x, (-1, 30, 2048)), output_shape=(30, 2048))(expanded_spatial_features)

temporal_features = build_temporal_feature_extractor()(reshaped_spatial_features)

micro_exp_spatial_features = build_micro_exp_spatial_feature_extractor()(micro_exp_spatial_input)

expanded_micro_spatial_features = Lambda(lambda x: tf.expand_dims(x, axis=1))(micro_exp_spatial_features)
reshaped_micro_spatial_features = Lambda(lambda x: tf.reshape(x, (-1, 30, 128)), output_shape=(30, 128))(expanded_micro_spatial_features)

micro_exp_temporal_features = build_temporal_inconsistency_attention()(reshaped_micro_spatial_features)


In [30]:
fused_features = build_feature_fusion_layer(
    spatial_features, 
    temporal_features, 
    micro_exp_spatial_features,
    micro_exp_temporal_features
)

In [31]:
final_model = build_face_swap_detection_model(fused_features)

In [32]:
#from function import load_dataset

In [33]:
#video_data_test1 = load_dataset()

In [34]:
#video_data_test1

In [35]:
#video_dataframe = pd.DataFrame(video_data_test1)

In [36]:
#video_dataframe.head(5)

In [37]:
#video_dataframe = video_dataframe.transpose()

In [38]:
#video_dataframe.columns

In [39]:
#type(video_dataframe['frames'][0][0])

In [40]:
#video_dataframe.index.name = "video_index"

In [41]:
#video_dataframe

In [42]:
# #def assign_video_label(frame_labels, micro_expression_labels):
#     # Any frame or micro-expression being manipulated sets video as manipulated (1)
#     if np.any(np.array(frame_labels) == 1) or np.any(np.array(micro_expression_labels) == 1):
#         return 1
#     else:
#         return 0

In [43]:
# video_dataframe['video_label'] = video_dataframe.apply(
#     lambda row: assign_video_label(row['frame_label'], row['Micro_Expression_label']),
#     axis=1
# )

In [44]:
# video_dataframe.head(1)

In [45]:
# type(video_dataframe['frames'][0][0])

In [46]:
# import pickle
# import pandas as pd
# from PIL import Image
# from io import BytesIO

# # Function to convert PIL images to byte arrays
# def pil_to_bytes(pil_img):
#     with BytesIO() as buffer:
#         pil_img.save(buffer, format='JPEG')
#         return buffer.getvalue()

# # Create a dictionary to hold the video data
# pickled_data = {}

# # Iterate over DataFrame rows
# for idx, row in video_dataframe.iterrows():
#     video_name = idx  # or another unique identifier if applicable
#     pickled_data[video_name] = {
#         'frames': [pil_to_bytes(img) for img in row['frames']],
#         'frame_label': row['frame_label'],
#         'Micro_Expression': [pil_to_bytes(img) for img in row['Micro_Expression']],
#         'Micro_Expression_label': row['Micro_Expression_label']
#     }

# # Save the dictionary to a pickle file
# with open('video_data_2.pkl', 'wb') as f:
#     pickle.dump(pickled_data, f)

# print("Data saved to pickle format.")


In [47]:
import pickle
from PIL import Image
from io import BytesIO
import pandas as pd

# Function to convert byte arrays back to PIL images
def bytes_to_pil(byte_data):
    with BytesIO(byte_data) as buffer:
        return Image.open(buffer)

# Load data from pickle file
with open('video_data_2.pkl', 'rb') as f:
    loaded_data = pickle.load(f)

# Create a list to hold the restored data
restored_data = []

# Reconstruct the DataFrame-like structure
for video_name, video_info in loaded_data.items():
    restored_data.append({
        'video_name': video_name,
        'frames': [bytes_to_pil(img_bytes) for img_bytes in video_info['frames']],
        'frame_label': video_info['frame_label'],
        'Micro_Expression': [bytes_to_pil(img_bytes) for img_bytes in video_info['Micro_Expression']],
        'Micro_Expression_label': video_info['Micro_Expression_label']
    })

# Convert to DataFrame if needed
restored_dataframe = pd.DataFrame(restored_data)

print("Data loaded and restored.")


Data loaded and restored.


In [48]:
type(restored_dataframe['frames'][0][0])

PIL.JpegImagePlugin.JpegImageFile

In [49]:
from tensorflow.keras.optimizers import Adam

# Compile the model
final_model.compile(optimizer=Adam(), loss='binary_crossentropy', metrics=['accuracy']) # Adjust epochs as needed


In [50]:
from datamaker import VideoDataGenerator

In [51]:
import pickle

# Load the data from the pickle file
with open('video_data_2.pkl', 'rb') as f:
    pickled_data = pickle.load(f)

In [52]:
from sklearn.model_selection import train_test_split

# Convert your video_data dictionary to a list of items for easier splitting
data_items = list(pickled_data.items())
video_names, labels = zip(*[(video_name, video_info['frame_label'][0]) for video_name, video_info in pickled_data.items()])

# Split the data
train_names, temp_names, train_labels, temp_labels = train_test_split(video_names, labels, test_size=0.3, random_state=42)
val_names, test_names, val_labels, test_labels = train_test_split(temp_names, temp_labels, test_size=0.5, random_state=42)

# Prepare dictionaries for each split
train_data = {name: pickled_data[name] for name in train_names}
val_data = {name: pickled_data[name] for name in val_names}
test_data = {name: pickled_data[name] for name in test_names}



In [53]:
# Define the output signature for the generator
output_signature = (
    (
        tf.TensorSpec(shape=(None, 30, 224, 224, 3), dtype=tf.float32),
        tf.TensorSpec(shape=(None, 30, 64, 64, 3), dtype=tf.float32)
    ),
    tf.TensorSpec(shape=(None,), dtype=tf.float32)
)

In [54]:
train_generator = tf.data.Dataset.from_generator(
    lambda: VideoDataGenerator(train_data),
    output_signature=output_signature
)

In [55]:
val_generator = tf.data.Dataset.from_generator(
    lambda: VideoDataGenerator(val_data),
    output_signature=output_signature
)

In [56]:
test_generator = tf.data.Dataset.from_generator(
    lambda: VideoDataGenerator(test_data),
    output_signature=output_signature
)

In [57]:
# Inputs
facial_input = Input(shape=(30, 224, 224, 3), name='frames_input')
micro_exp_inputs = Input(shape=(30, 64, 64, 3), name='micro_exp_input')

In [58]:
# Define the final model
dfds_model = Model(inputs=[facial_input, micro_exp_inputs], outputs=final_model.output)

# Compile the model
dfds_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


In [59]:
dfds_model.input_shape

[(None, 30, 224, 224, 3), (None, 30, 64, 64, 3)]

In [60]:
train_generator

<_FlatMapDataset element_spec=((TensorSpec(shape=(None, 30, 224, 224, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None, 30, 64, 64, 3), dtype=tf.float32, name=None)), TensorSpec(shape=(None,), dtype=tf.float32, name=None))>

In [11]:
import pickle
from datamaker import VideoDataGenerator
import tensorflow as tf
from deliver import deliver_model
from tensorflow import keras


with open('../data/video_data_2.pkl', 'rb') as f:
    pickled_data = pickle.load(f)

from sklearn.model_selection import train_test_split

# Convert your video_data dictionary to a list of items for easier splitting
data_items = list(pickled_data.items())
video_names, labels = zip(*[(video_name, video_info['frame_label'][0]) for video_name, video_info in pickled_data.items()])

# Split the data
train_names, temp_names, train_labels, temp_labels = train_test_split(video_names, labels, test_size=0.3, random_state=42)
val_names, test_names, val_labels, test_labels = train_test_split(temp_names, temp_labels, test_size=0.5, random_state=42)

# Prepare dictionaries for each split
train_data = {name: pickled_data[name] for name in train_names}
val_data = {name: pickled_data[name] for name in val_names}
test_data = {name: pickled_data[name] for name in test_names}

# Define the output signature for the generator
output_signature = (
    (
        tf.TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32),
        tf.TensorSpec(shape=(None, 64, 64, 3), dtype=tf.float32)
    ),
    tf.TensorSpec(shape=(None,1), dtype=tf.float32)
)

train_generator = tf.data.Dataset.from_generator(
    lambda: VideoDataGenerator(train_data),
    output_signature=output_signature
)

val_generator = tf.data.Dataset.from_generator(
    lambda: VideoDataGenerator(val_data),
    output_signature=output_signature
)

test_generator = tf.data.Dataset.from_generator(
    lambda: VideoDataGenerator(test_data),
    output_signature=output_signature
)

In [6]:
from deliver import deliver_model

In [7]:
model_test_1 = deliver_model()

In [12]:
model_test_1.compile(optimizer='adam', loss='binary_crossentropy', metrics=[keras.metrics.BinaryAccuracy(name="accuracy")])

In [13]:
model_test_1.summary()

Model: "functional_4"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_1       │ (None, 64, 64, 3) │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ functional          │ (None, 2048)      │ 23,587,712 │ input_layer[0][0] │
│ (Functional)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ functional_1        │ (None, 128)       │  2,224,448 │ input_layer_1[0]… │
│ (Functional)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lambda (Lambda)     │ (None, 1, 2048)   │          0 │ functional[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lambda_2 (Lambda)   │ (None, 1, 128)    │          0 │ functional_1[0][… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lambda_1 (Lambda)   │ (None, 30, 2048)  │          0 │ lambda[0][0]      │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lambda_3 (Lambda)   │ (None, 30, 128)   │          0 │ lambda_2[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ functional_2        │ (None, 30, 128)   │  2,393,600 │ lambda_1[0][0]    │
│ (Functional)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lambda_6 (Lambda)   │ (None, 1, 1, 128) │          0 │ functional_1[0][… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ functional_3        │ (None, 30, 128)   │    493,440 │ lambda_3[0][0]    │
│ (Functional)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_4 (Dense)     │ (None, 30, 1)     │        129 │ functional_2[0][… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_4 (Conv2D)   │ (None, 1, 1, 1)   │        129 │ lambda_6[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_5 (Dense)     │ (None, 30, 1)     │        129 │ functional_3[0][… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_4        │ (None, 30, 1)     │          0 │ dense_4[0][0]     │
│ (Activation)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_5        │ (None, 1, 1, 1)   │          0 │ conv2d_4[0][0]    │
│ (Activation)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_6        │ (None, 30, 1)     │          0 │ dense_5[0][0]     │
│ (Activation)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lambda_4 (Lambda)   │ (None, 1, 1,      │          0 │ functional[0][0]  │
│                     │ 2048)             │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ multiply_1          │ (None, 30, 128)   │          0 │ functional_2[0][… │
│ (Multiply)          │                   │            │ activation_4[0][… │
├─────────────────────┼───────────────────┼────────────┼─────────────────

 Total params: 29,363,652 (112.01 MB)

 Trainable params: 5,775,492 (22.03 MB)

 Non-trainable params: 23,588,160 (89.98 MB)

In [14]:
model_test_1.fit(
    train_generator,
    epochs=10,
    verbose=1,
    validation_data=val_generator
)

Epoch 1/10


ValueError: Cannot get result() since the metric has not yet been built.